In [1]:
from trainer import *
import yaml

In [28]:
# create a tensor with random numbers between 0 and 1 of size (33945, 1)
# x = 0.5 + torch.rand(33945, 1)
seed = 28
np.random.seed(seed)
sample_functions = {False: np.random.uniform, True: np.random.randint}
# sample uniformly from the range (discrete)
discrete = False
# this_range = [4, 7]
this_range = [6.3, 11.7]
this_sample_function = sample_functions[discrete]
x = torch.tensor(this_sample_function(*this_range, 33945)).unsqueeze(1)
# x = torch.tensor(this_sample_function(*this_range, 33945)).unsqueeze(1)
# x = 0.5 + torch.rand(33945, 1)
print(x)
# torch.save(x, 'data_files/favorita/random_lead_times.pt')
torch.save(x, 'data_files/favorita/random_underage_costs.pt')

tensor([[10.2367],
        [ 9.3307],
        [ 6.9748],
        ...,
        [11.5261],
        [ 9.3131],
        [ 7.4145]], dtype=torch.float64)


In [4]:
# read a tensor .pt file
df = pd.read_csv('data_files/favorita//tensors_row_info.csv')

In [6]:
print(df.head())
print(len(df))

   store_nbr  item_nbr     family  class  perishable
0          1     99197  GROCERY I   1067           0
1          1    103520  GROCERY I   1028           0
2          1    108634  GROCERY I   1075           0
3          1    108797  GROCERY I   1004           0
4          1    108862  GROCERY I   1062           0
33945


In [2]:
common_data = pd.read_csv(f'data_files/favorita/dates_with_info.csv')

In [7]:
christmas = torch.tensor(common_data['days_from_christmas'])

In [8]:
print(christmas)

tensor([   7,   14,   21,   28,   35,   42,   49,   56,   63,   70,   77,   84,
          91,   98,  105,  112,  119,  126,  133,  140,  147,  154,  161,  168,
         175,  182, -176, -169, -162, -155, -148, -141, -134, -127, -120, -113,
        -106,  -99,  -92,  -85,  -78,  -71,  -64,  -57,  -50,  -43,  -36,  -29,
         -22,  -15,   -8,   -1,    6,   13,   20,   27,   34,   41,   48,   55,
          62,   69,   76,   83,   90,   97,  104,  111,  118,  125,  132,  139,
         146,  153,  160,  167,  174,  181, -177, -170, -163, -156, -149, -142,
        -135, -128, -121, -114, -107, -100,  -93,  -86,  -79,  -72,  -65,  -58,
         -51,  -44,  -37,  -30,  -23,  -16,   -9,   -2,    5,   12,   19,   26,
          33,   40,   47,   54,   61,   68,   75,   82,   89,   96,  103,  110,
         117,  124,  131,  138,  145,  152,  159,  166,  173,  180, -178, -171,
        -164, -157, -150, -143, -136, -129, -122, -115, -108, -101,  -94,  -87,
         -80,  -73,  -66,  -59,  -52,  -

In [2]:
with open('config_files/one_store_backlogged.yml', 'r') as file:
    config = yaml.safe_load(file)
keys = 'seeds', 'shifted_seeds', 'problem_params', 'params_by_dataset', 'trainer_params', 'observation_params', 'store_params', 'warehouse_params', 'optimizer_params', 'nn_params'
seeds, shifted_seeds, problem_params, params_by_dataset, trainer_params, observation_params, store_params, warehouse_params, optimizer_params, nn_params = [config[key] for key in keys]

In [3]:
print(config.keys())
print(f'{config["echelon_params"]}')
print(f'{config["store_params"]["lead_time"]}')

dict_keys(['seeds', 'shifted_seeds', 'problem_params', 'params_by_dataset', 'observation_params', 'store_params', 'warehouse_params', 'echelon_params', 'trainer_params', 'optimizer_params', 'nn_params'])
None
{'sample_across_stores': False, 'vary_across_samples': False, 'expand': True, 'value': 4}


In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
creator = DatasetCreator()

scenario = Scenario(
    params_by_dataset['train']['periods'], problem_params, store_params, warehouse_params, 
    params_by_dataset['train']['n_samples'] + params_by_dataset['dev']['n_samples'], seeds
    )

train_dataset, dev_dataset = creator.create_datasets(scenario, split=True, by_sample_indexes=True, sample_index_for_split=params_by_dataset['dev']['n_samples'])
train_loader = DataLoader(train_dataset, batch_size=params_by_dataset['train']['batch_size'], shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=params_by_dataset['dev']['batch_size'], shuffle=False)

scenario = Scenario(params_by_dataset['test']['periods'], problem_params, store_params, warehouse_params, params_by_dataset['test']['n_samples'], shifted_seeds)
test_dataset = creator.create_datasets(scenario, split=False)
test_loader = DataLoader(test_dataset, batch_size=params_by_dataset['test']['batch_size'], shuffle=False)

data_loaders = {'train': train_loader, 'dev': dev_loader, 'test': test_loader}

model = get_architecture(nn_params['name'])(nn_params, output_size=problem_params['n_stores'], device=device).to(device)
# model = FullyConnectedNN(nn_params, output_size=problem_params['n_stores']).to(device)
loss_function = PolicyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_params['learning_rate'])

simulator = Simulator(device=device)
trainer = Trainer(device=device)

/user/ma4177/.conda/envs/MatiasRL/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
print(device)

cuda:0


In [5]:
trainer.train(trainer_params['epochs'], loss_function, simulator, model, data_loaders, optimizer, problem_params, observation_params, params_by_dataset, trainer_params)

KeyboardInterrupt: 

In [7]:
average_test_loss, average_test_loss_to_report = trainer.test(
    loss_function, simulator, model, data_loaders, optimizer, problem_params, observation_params, params_by_dataset, discrete_demand=False
    )

In [8]:
print(f'test loss: {average_test_loss_to_report}')

test loss: 5.658812561035156


In [ ]:
# plot the losses
ymin, ymax = 0, 100  # set the y-axis limit for the plot
trainer.plot_losses(ymin=ymin, ymax=ymax)